# Fine-tuning BERT for Question Answering with SQuAD

## Paso 1: Instalación e importación de librerías necesarias

In [1]:
# Instalación de librerías
!pip install transformers accelerate datasets trl bitsandbytes -qU


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 58.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

In [2]:
# Importación de librerías
import torch
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments,BertTokenizerFast
from datasets import load_dataset, load_metric

## Paso 2: Carga y preprocesamiento de datos

In [3]:
# Cargar el conjunto de datos SQuAD
dataset = load_dataset("squad")

# Tokenización de los datos utilizando BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Encuentra el índice de inicio del token
        context_start = sequence_ids.index(1)
        context_end = 1 + len([x for x in sequence_ids if x == 1])

        # Ajuste de posición
        token_start_index = inputs.char_to_token(i, start_char)
        token_end_index = inputs.char_to_token(i, end_char - 1)

        if token_start_index is None:
            token_start_index = context_start
        if token_end_index is None:
            token_end_index = context_end - 1

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    inputs.update({
        "start_positions": start_positions,
        "end_positions": end_positions
    })
    return inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

## Paso 3: Configuración y carga del modelo preentrenado

In [4]:
# Cargar el modelo preentrenado BERT
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Paso 4: Fine-tuning del modelo con los datos SQuAD

In [5]:
# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    max_steps=500,
    learning_rate=2e-4,
    save_total_limit=3,
    logging_steps=10,
    output_dir="./results",
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    evaluation_strategy="steps",
    eval_steps=50,
    warmup_ratio=0.05,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
# Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

max_steps is given, it will override any value given in num_train_epochs


In [7]:
# Entrenar el modelo
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,2.307500,2.244938
100,1.814400,1.854336
150,1.449900,1.527578
200,1.662500,1.577577
250,1.397800,1.561349
300,1.420800,1.448287
350,1.161100,1.253287
400,1.227700,1.103173
450,1.179400,1.030502
500,1.102100,1.014567


TrainOutput(global_step=500, training_loss=1.681499418258667, metrics={'train_runtime': 2993.545, 'train_samples_per_second': 2.672, 'train_steps_per_second': 0.167, 'total_flos': 1567780540416000.0, 'train_loss': 1.681499418258667, 'epoch': 0.091324200913242})

## Paso 5: Evaluación del modelo

In [8]:
# Evaluar el modelo
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 1.0145667791366577, 'eval_runtime': 225.1586, 'eval_samples_per_second': 46.945, 'eval_steps_per_second': 5.871, 'epoch': 0.091324200913242}


## Paso 6: Guardado y compartición del modelo en Hugging Face

In [15]:
from huggingface_hub import notebook_login

# Autenticarse con Hugging Face
notebook_login()

# Guardar el modelo en Hugging Face
model.push_to_hub("Fine_tuning_BERT_SQuAD", use_auth_token=True)
tokenizer.push_to_hub("Fine_tuning_BERT_SQuAD", use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:836: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CMolano/Fine_tuning_BERT_SQuAD/commit/2373f3ef32dd867ecf51e42853adf8d7f8769276', commit_message='Upload tokenizer', commit_description='', oid='2373f3ef32dd867ecf51e42853adf8d7f8769276', pr_url=None, pr_revision=None, pr_num=None)